In [124]:
import pandas as pd
fa17 = pd.read_csv('fa17/p_co.csv')
fa18 = pd.read_csv('fa18/p_co.csv')
fa19 = pd.read_csv('fa19/p_co.csv')
#fa20 = pd.read_csv('/home/asj53/data/fa20/p_co.csv')
fa21 = pd.read_csv('fa21/coenrollment_matrix.csv')
fa22 = pd.read_csv('fa22/coenrollment_matrix.csv')
fa23 = pd.read_csv('fa23/p_co.csv')
fa24 = pd.read_csv('fa24/p_co.csv')
sp17 = pd.read_csv('sp17/p_co.csv')
sp18 = pd.read_csv('sp18/p_co.csv')
sp19 = pd.read_csv('sp19/p_co.csv')
#sp17 = pd.read_csv('sp20/p_co.csv')
#sp17 = pd.read_csv('sp21/p_co.csv')
sp22 = pd.read_csv('sp22/coenrollment_matrix.csv')
sp23 = pd.read_csv('sp23/coenrollment_matrix.csv')
sp24 = pd.read_csv('sp24/p_co.csv')
sp25 = pd.read_csv('sp25/p_co.csv')
sp25

,Unnamed: 0,1R-0001C,1R-0003C,1R-0008C,1R-0022C,1R-0027C,1R-0031C,1R-0033C,1R-0037C,1R-0053C,...,MLG65,MLG67,MLG68,MLG69,MLG72,MLG73,MLG75,MLG78,PR-16454,PR-16616
0,1R-0001C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1R-0003C,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1R-0008C,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,2,0,0
3,1R-0022C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1R-0027C,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,MLG73,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
566,MLG75,0,0,1,0,0,0,0,1,1,...,2,0,0,3,0,0,0,0,0,0
567,MLG78,0,0,2,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
568,PR-16454,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
import glob
import os
import numpy as np
import pandas as pd
from functools import reduce

# 1) A fully vectorised cleanup for codes
def clean_codes(idx: pd.Index) -> pd.Index:
    s = idx.astype(str)
    # drop any 'R' in the first 5 chars, then strip trailing 'C'
    first5 = s.str[:5].str.replace('R', '', regex=False)
    out    = (first5 + s.str[5:]).str.rstrip('C')
    return out

# 2) Load & clean each semester into a dict of small (n×n) DataFrames
def load_coenrollment(globs):
    sems = {}
    for pattern in globs:
        for fp in glob.glob(pattern):
            sem = os.path.basename(os.path.dirname(fp))  # e.g. "fa17" from ".../fa17/p_co.csv"
            df = pd.read_csv(fp, index_col=0).fillna(0).astype(int)
            
            # clean & collapse duplicate codes
            df.index   = clean_codes(df.index)
            df = df.groupby(df.index).sum()
            df.columns  = clean_codes(df.columns)
            df = df.groupby(axis=1, level=0).sum()
            
            sems[sem] = df
    return sems

patterns = [
    #"fa*/p_co.csv","fa*/coenrollment_matrix.csv",
    "sp*/p_co.csv","sp*/coenrollment_matrix.csv",
]
semester_dfs = load_coenrollment(patterns)


# 3) For each semester, pull out only the upper‐triangle non‐zero co‐enrolments into a long DataFrame
long_records = []
for sem, mat in semester_dfs.items():
    arr  = mat.values
    mask = np.triu(np.ones_like(arr, dtype=bool), k=1)
    # find all (i,j) where count > 0 and i<j
    rows, cols = np.where((arr>0) & mask)
    long_records.append(pd.DataFrame({
        "course1": mat.index.values[rows],
        "course2": mat.columns.values[cols],
        "semester": sem,
        "count":     arr[rows, cols]
    }))

long_df = pd.concat(long_records, ignore_index=True)


# 4) Pivot to get a MultiIndex × semesters table, filling zeros where missing
coenrol = long_df.pivot_table(
    index=["course1","course2"],
    columns="semester",
    values="count",
    fill_value=0
)



/var/folders/sn/x9r30ljs78s79jhltsxlx8tw0000gn/T/ipykernel_7750/687512597.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).sum()
/var/folders/sn/x9r30ljs78s79jhltsxlx8tw0000gn/T/ipykernel_7750/687512597.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).sum()
/var/folders/sn/x9r30ljs78s79jhltsxlx8tw0000gn/T/ipykernel_7750/687512597.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).sum()
/var/folders/sn/x9r30ljs78s79jhltsxlx8tw0000gn/T/ipykernel_7750/687512597.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(axis=1, level=0).sum()
/var/folders/sn/x9r30ljs78s79jhltsxlx8tw0000gn/T/ipykernel_7750/687512597.py:27:

In [ ]:
def simulate_future_semester(coenrol, n_courses, m_samples, random_seed=None):
    """
    Simulate future semester co-enrollments based on historical data
    Uses greedy selection: pick first course randomly, then greedily select
    subsequent courses with highest co-enrollment potential with already selected courses.
    
    Parameters:
    - coenrol: DataFrame with MultiIndex (course1, course2) and semester columns
    - n_courses: number of courses to select
    - m_samples: total number of co-enrollment instances to draw
    - random_seed: for reproducibility
    
    Returns:
    - coenroll_matrix: symmetric co-enrollment matrix
    - selected_courses: list of selected courses
    - simulation_stats: dictionary with simulation statistics
    """
    if random_seed is not None:
        np.random.seed(random_seed)
        random.seed(random_seed)
    
    print(f"Starting simulation with {n_courses} courses and {m_samples} co-enrollment samples...")
    
    # Step 1: Get all unique courses
    all_courses = set()
    for course1, course2 in coenrol.index:
        all_courses.add(course1)
        all_courses.add(course2)
    all_courses = list(all_courses)
    
    if n_courses > len(all_courses):
        print(f"Warning: Requested {n_courses} courses but only {len(all_courses)} available. Using all courses.")
        n_courses = len(all_courses)
    
    # Sum across all semesters to get total historical co-enrollments
    historical_totals = coenrol.sum(axis=1)
    
    # Step 2: Greedy course selection
    selected_courses = []
    remaining_courses = all_courses.copy()
    
    # Pick first course randomly
    first_course = random.choice(remaining_courses)
    selected_courses.append(first_course)
    remaining_courses.remove(first_course)
    print(f"First course selected randomly: {first_course}")
    
    # Greedily select remaining courses
    for i in range(1, n_courses):
        if not remaining_courses:
            break
            
        best_course = None
        best_total_coenrollment = -1
        
        # For each remaining course, calculate total co-enrollment with selected courses
        for candidate in remaining_courses:
            total_coenrollment = 0
            
            for selected in selected_courses:
                # Check both orderings
                if (selected, candidate) in historical_totals.index:
                    count = historical_totals[(selected, candidate)]
                elif (candidate, selected) in historical_totals.index:
                    count = historical_totals[(candidate, selected)]
                else:
                    count = 0
                total_coenrollment += count
            
            # Update best if this candidate is better
            if total_coenrollment > best_total_coenrollment:
                best_total_coenrollment = total_coenrollment
                best_course = candidate
            elif total_coenrollment == best_total_coenrollment and best_course is None:
                # If tied at 0 (or first candidate), pick this one
                best_course = candidate
        
        # Add best course to selection
        if best_course is not None:
            selected_courses.append(best_course)
            remaining_courses.remove(best_course)
            print(f"Course {i+1} selected: {best_course} (total co-enrollment: {best_total_coenrollment})")
        else:
            print(f"Warning: Could not find candidate for position {i+1}")
            break
    
    print(f"Selected {len(selected_courses)} courses through greedy selection")
    
    # Step 3: Calculate historical co-enrollment probabilities for selected courses
    course_pairs = list(combinations(selected_courses, 2))
    print(f"Total possible course pairs: {len(course_pairs)}")
    
    # Get historical counts for each pair
    pair_counts = {}
    pairs_with_history = 0
    
    for course1, course2 in course_pairs:
        # Check both orderings since the data might be stored either way
        if (course1, course2) in historical_totals.index:
            count = historical_totals[(course1, course2)]
        elif (course2, course1) in historical_totals.index:
            count = historical_totals[(course2, course1)]
        else:
            count = 0  # No historical co-enrollment
        
        pair_counts[(course1, course2)] = count
        if count > 0:
            pairs_with_history += 1
    
    print(f"Course pairs with historical co-enrollment data: {pairs_with_history}/{len(course_pairs)}")
    
    # Convert to probabilities
    total_historical_count = sum(pair_counts.values())
    if total_historical_count == 0:
        # If no historical data, use uniform distribution
        probabilities = np.ones(len(course_pairs)) / len(course_pairs)
        print("Warning: No historical co-enrollment data found for selected courses. Using uniform distribution.")
    else:
        probabilities = np.array([pair_counts[pair] for pair in course_pairs]) / total_historical_count
    
    # Step 4: Sample m co-enrollments based on probabilities
    sampled_indices = np.random.choice(
        len(course_pairs), 
        size=m_samples, 
        p=probabilities
    )
    
    # Count occurrences of each pair
    simulated_counts = {}
    for i in sampled_indices:
        pair = course_pairs[i]
        simulated_counts[pair] = simulated_counts.get(pair, 0) + 1
    
    # Step 5: Create symmetric co-enrollment matrix
    coenroll_matrix = pd.DataFrame(
        0, 
        index=selected_courses, 
        columns=selected_courses
    )
    
    for (course1, course2), count in simulated_counts.items():
        coenroll_matrix.loc[course1, course2] = count
        coenroll_matrix.loc[course2, course1] = count  # Make symmetric
    
    # Calculate statistics
    total_coenrollments = sum(simulated_counts.values())
    unique_pairs_with_coenrollments = len(simulated_counts)
    max_coenrollments = max(simulated_counts.values()) if simulated_counts else 0
    
    simulation_stats = {
        'total_samples': m_samples,
        'total_coenrollments': total_coenrollments,
        'unique_pairs_with_coenrollments': unique_pairs_with_coenrollments,
        'max_coenrollments_per_pair': max_coenrollments,
        'courses_selected': len(selected_courses),
        'pairs_with_historical_data': pairs_with_history
    }
    
    return coenroll_matrix, selected_courses, simulation_stats

# To run with your data, use:
#simulated_matrix, courses, stats = run_simulation_example(coenrol)

# Or customize your own simulation:
simulated_matrix, courses, stats = simulate_future_semester(
     coenrol, 
     n_courses=600,    # Pick 20 random courses
     m_samples=100000,  # Draw 1000 co-enrollment samples
     random_seed=42   # For reproducible results
 )

Starting simulation with 600 courses and 100000 co-enrollment samples...
Selected 600 courses: ['1-10959', '1-9954', '1-7993', '1-9907', '1-17855', '1-7622', '1-10766', '1-8314', '1-5965', '1-17790']...
Total possible course pairs: 179700
Course pairs with historical co-enrollment data: 3756/179700


In [117]:
simulated_matrix.to_csv('large_p_co.csv')

In [119]:
mid_sim, courses, stats = simulate_future_semester(
     coenrol, 
     n_courses=400,    # Pick 20 random courses
     m_samples=60000,  # Draw 1000 co-enrollment samples
     random_seed=42   # For reproducible results
 )
mid_sim.to_csv('mid_p_co.csv')

Starting simulation with 400 courses and 60000 co-enrollment samples...
Selected 400 courses: ['1-10959', '1-9954', '1-7993', '1-9907', '1-17855', '1-7622', '1-10766', '1-8314', '1-5965', '1-17790']...
Total possible course pairs: 79800
Course pairs with historical co-enrollment data: 1752/79800


In [120]:
small_sim, courses, stats = simulate_future_semester(
     coenrol, 
     n_courses=200,    # Pick 20 random courses
     m_samples=20000,  # Draw 1000 co-enrollment samples
     random_seed=42   # For reproducible results
 )
small_sim.to_csv('mid_p_co.csv')

Starting simulation with 200 courses and 20000 co-enrollment samples...
Selected 200 courses: ['1-10959', '1-9954', '1-7993', '1-9907', '1-17855', '1-7622', '1-10766', '1-8314', '1-5965', '1-17790']...
Total possible course pairs: 19900
Course pairs with historical co-enrollment data: 541/19900
